In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import keras
import pickle
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


# Recover model, tokenizer and label encoder

In [2]:
model = keras.models.load_model('model.h5')

with open('tokenizer.json', 'rb') as f:
    s = f.read()
    t = keras.preprocessing.text.tokenizer_from_json(s)

with open('label_encoder.pkl', 'rb') as f:
    le = pickle.load(f)

/home/tomek/anaconda3/envs/cmpe255/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [75]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 64)           1920000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 98, 250)           48250     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 250)               62750     
_________________________________________________________________
dense_2 (Dense)              (None, 20)                5020      
Total params: 2,036,020
Trainable params: 2,036,020
Non-trainable params: 0
_________________________________________________________________


In [38]:
def who_would_say_that(s):
    """
    s is a string of at least 100 words
    returns prediction with the highest score
    """
    if(len(s)<100): return None
    seq = t.texts_to_sequences([s])[0][:100]
    batch = np.zeros((64,100))
    batch[0] = seq
    scores = model.predict([batch], batch_size=1)
    return le.inverse_transform([scores[0].argmax()])

def classify(s):
    """
    s is a string of at least 100 words
    returns a dict in form of {'author': score}
    """
    
    if(len(s)<100): return None
    seq = t.texts_to_sequences([s])[0][:100]
    batch = np.zeros((64,100))
    batch[0] = seq
    scores = model.predict([batch], batch_size=1)
    return {c: f"{100 * round(scores[0][idx],4)} %" for idx, c in enumerate(le.classes_)}

def get_raw_scores(s):
    """
    s is a string of at least 100 words
    returns a dict in form of {'author': score}
    """
    
    if(len(s)<100): return None
    seq = t.texts_to_sequences([s])[0][:100]
    batch = np.zeros((64,100))
    batch[0] = seq
    scores = model.predict([batch], batch_size=1)
    return {c: scores[0][idx] for idx, c in enumerate(le.classes_)}

In [62]:

# https://www.cbc.ca/books/void-between-worlds-by-vaishnavi-jagadeeswaran-1.5149115
shakespeare_style = """
O knaves embolden'd! Lend your ears to me,
Of this foul gloom of mine be drawn towards, 
Let it be known, of its position poised
This heart of mine, it looms darkly over, 
And stirs a grievous sadness now awake.

Ethereal spires hath come aflame, alight!
How burned by devilish industry; lo,
The bells of brass and bronze sing sweet no more,
For they lay silent, trapped within their tomb,
Made quiet by the blaze, and wounded by the heat.

How overwrought by grief, how sorrowful,
Take flight, o soulless arch-villain so near,
Be not a shroud lain over I, begone!

And yet, o my Bertram, loved by me,
He shirks his duty to grieve as all do. 
Share not with I does he my thoughts of it, 
Instead he goes about his work becalmed! 
Perhaps I ought to be assured he cares, 
And he has taken pains to becalm me.
Yes, that is what he must be doing now,
Adore his sweetly caring heart do I. 

Alack, for now I hear the choir song,
Upon the cobblestones agleam with rain, 
They sing their words of God's forgiving grace, 
As all of Paris dreams and weeps as one, 
Awaiting news of our cathedral old,
And what we all must do to aid our love.
"""

marlow = """
CHORUS. Not marching now in fields of Thrasymene,
     Where Mars did mate1 the Carthaginians;
     Nor sporting in the dalliance of love,
     In courts of kings where state is overturn'd;
     Nor in the pomp of proud audacious deeds,
     Intends our Muse to vaunt2 her3 heavenly verse:
     Only this, gentlemen,—we must perform
     The form of Faustus' fortunes, good or bad:
     To patient judgments we appeal our plaud,
     And speak for Faustus in his infancy.
     Now is he born, his parents base of stock,
     In Germany, within a town call'd Rhodes:
     Of riper years, to Wertenberg he went,
     Whereas4 his kinsmen chiefly brought him up.
     So soon he profits in divinity,
     The fruitful plot of scholarism grac'd,
     That shortly he was grac'd with doctor's name,
     Excelling all whose sweet delight disputes
     In heavenly matters of theology;
     Till swoln with cunning,5 of a self-conceit,
     His waxen wings did mount above his reach,
     And, melting, heavens conspir'd his overthrow;
     For, falling to a devilish exercise,
"""


actual_dickens = """
An aunt of my father’s, and consequently a great-aunt of mine, of whom
I shall have more to relate by and by, was the principal magnate of our
family. Miss Trotwood, or Miss Betsey, as my poor mother always called
her, when she sufficiently overcame her dread of this formidable
personage to mention her at all (which was seldom), had been married
to a husband younger than herself, who was very handsome, except in the
sense of the homely adage, ‘handsome is, that handsome does’--for he
was strongly suspected of having beaten Miss Betsey, and even of having once,
on a disputed question of supplies, made some hasty but determined
arrangements to throw her out of a two pair of stairs’ window"""

# Classify Shakespeare

In [67]:
actual_shakespeare = """To be, or not to be: that is the question: Whether 'tis nobler in the mind to suffer The slings and arrows of outrageous fortune, Or to take arms against a sea of troubles, And by opposing end them? To die: to sleep; No more; and by a sleep to say we end The heart-ache and the thousand natural shocks That flesh is heir to, 'tis a consummation Devoutly to be wish'd. To die, to sleep; To sleep: perchance to dream: ay, there's the rub; For in that sleep of death what dreams may come When we have shuffled off this mortal coil, Must give us pause: there's the respect That makes calamity of so long life; For who would bear the whips and scorns of time, The oppressor's wrong, the proud man's contumely, The pangs of despised love, the law's delay,"""
who_would_say_that(actual_shakespeare)

array(['Shakespeare, William'], dtype='<U35')

# Donald Trump's latest tweet

In [74]:
realdonaldtrump = """The White House CoronaVirus Task Force, headed by Vice President Mike Pence, has done a fantastic job of bringing together vast highly complex resources that have set a high standard for others to follow in the future. Ventilators, which were few & in bad shape, are now being produced in the thousands, and we have many to spare. We are helping other countries which are desperate for them. Likewise, after having been left little, we are now doing more testing than all other countries combined, and with superior tests. Face masks & shields, gloves, gowns etc. are now plentiful. The last four Governors teleconference calls have been conclusively strong. Because of this success, the Task Force will continue on indefinitely with its focus on SAFETY & OPENING UP OUR COUNTRY AGAIN. We may add or subtract people to it, as appropriate. The Task Force will also be very focused on Vaccines & Therapeutics. Thank you!"""
get_percentages(realdonaldtrump)

{'Alger, Horatio, Jr.': '0.0 %',
 'Ballantyne, R. M. (Robert Michael)': '0.0 %',
 'Balzac, Honoré de': '0.0 %',
 'Dickens, Charles': '0.0 %',
 'Ebers, Georg': '0.0 %',
 'Fenn, George Manville': '0.0 %',
 'Hawthorne, Nathaniel': '0.0 %',
 'Henty, G. A. (George Alfred)': '0.0 %',
 'Howells, William Dean': '0.0 %',
 'Jacobs, W. W. (William Wymark)': '0.0 %',
 'Kingston, William Henry Giles': '0.0 %',
 'Lytton, Edward Bulwer Lytton, Baron': '0.0 %',
 'Meredith, George': '0.0 %',
 'Motley, John Lothrop': '0.0 %',
 'Oliphant, Mrs. (Margaret)': '0.0 %',
 'Parker, Gilbert': '0.0 %',
 'Pepys, Samuel': '0.0 %',
 'Shakespeare, William': '0.0 %',
 'Stevenson, Robert Louis': '0.0 %',
 'Twain, Mark': '0.0 %'}

# Classify Christopher Marlowe
Not in the datset, but __very__ close to Shakespeare (same era, similar style)

In [72]:
marlowe = """
CHORUS. Not marching now in fields of Thrasymene,
     Where Mars did mate1 the Carthaginians;
     Nor sporting in the dalliance of love,
     In courts of kings where state is overturn'd;
     Nor in the pomp of proud audacious deeds,
     Intends our Muse to vaunt2 her3 heavenly verse:
     Only this, gentlemen,—we must perform
     The form of Faustus' fortunes, good or bad:
     To patient judgments we appeal our plaud,
     And speak for Faustus in his infancy.
     Now is he born, his parents base of stock,
     In Germany, within a town call'd Rhodes:
     Of riper years, to Wertenberg he went,
     Whereas4 his kinsmen chiefly brought him up.
     So soon he profits in divinity,
     The fruitful plot of scholarism 
"""

who_would_say_that(marlowe)

array(['Shakespeare, William'], dtype='<U35')

In [73]:
get_percentages(marlowe)

{'Alger, Horatio, Jr.': '0.0 %',
 'Ballantyne, R. M. (Robert Michael)': '0.0 %',
 'Balzac, Honoré de': '0.0 %',
 'Dickens, Charles': '0.0 %',
 'Ebers, Georg': '0.0 %',
 'Fenn, George Manville': '0.0 %',
 'Hawthorne, Nathaniel': '0.0 %',
 'Henty, G. A. (George Alfred)': '0.0 %',
 'Howells, William Dean': '0.0 %',
 'Jacobs, W. W. (William Wymark)': '0.0 %',
 'Kingston, William Henry Giles': '0.0 %',
 'Lytton, Edward Bulwer Lytton, Baron': '0.0 %',
 'Meredith, George': '0.0 %',
 'Motley, John Lothrop': '0.0 %',
 'Oliphant, Mrs. (Margaret)': '0.0 %',
 'Parker, Gilbert': '0.0 %',
 'Pepys, Samuel': '0.0 %',
 'Shakespeare, William': '100.0 %',
 'Stevenson, Robert Louis': '0.0 %',
 'Twain, Mark': '0.0 %'}

In [66]:
shakespeare_style_competition = """
O knaves embolden'd! Lend your ears to me,
Of this foul gloom of mine be drawn towards, 
Let it be known, of its position poised
This heart of mine, it looms darkly over, 
And stirs a grievous sadness now awake.

Ethereal spires hath come aflame, alight!
How burned by devilish industry; lo,
The bells of brass and bronze sing sweet no more,
For they lay silent, trapped within their tomb,
Made quiet by the blaze, and wounded by the heat.

How overwrought by grief, how sorrowful,
Take flight, o soulless arch-villain so near,
Be not a shroud lain over I, begone!

And yet, o my Bertram, loved by me,
He shirks his duty to grieve as all do. 
Share not with I does he my thoughts of it, 
Instead he goes about his work becalmed! 
Perhaps I ought to be assured he cares, 
And he has taken pains to becalm me.
Yes, that is what he must be doing now,
Adore his sweetly caring heart do I. 

Alack, for now I hear the choir song,
Upon the cobblestones agleam with rain, 
They sing their words of God's forgiving grace, 
As all of Paris dreams and weeps as one, 
Awaiting news of our cathedral old,
And what we all must do to aid our love.
"""